In [2]:
import pyreadstat
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression

In [23]:
x_df, x_meta = pyreadstat.read_sas7bdat("../SASData2021/merged.sas7bdat")
x_df.head()

,gvkey,fyear,datadate,ACCHG,ACCRT,ACDO,ACO,ACODO,ACOMINC,ACOX,...,actual,CONSENSUS,IBESSHROUT,anndats,actual_p1,consensus_p1,ibesshrout_p1,AFE_p1,AFE,accruals
0,012994,1993.0,12234.0,0.000,NaN,NaN,0.916,NaN,NaN,0.916,...,1.180,1.40,3.255,1993-09-01,0.515,0.61,3.286,-0.011379,-0.026103,-0.006561
1,012994,1994.0,12599.0,0.092,NaN,NaN,0.837,NaN,NaN,0.837,...,0.515,0.61,3.286,1995-01-18,0.680,0.63,3.336,0.005937,-0.011111,-0.056665
2,012994,1995.0,12964.0,0.000,NaN,NaN,0.508,NaN,NaN,0.508,...,0.680,0.63,3.336,1995-09-14,0.950,0.55,3.535,0.044743,0.005278,-0.066196
3,012994,1996.0,13330.0,0.000,NaN,NaN,1.969,NaN,NaN,1.969,...,0.950,0.55,3.535,1997-07-21,0.550,0.55,3.567,0.000000,0.038571,0.023459
4,012994,2010.0,18627.0,0.000,NaN,NaN,5.258,NaN,0.047,NaN,...,0.920,1.05,7.834,2011-04-05,0.580,0.90,8.154,-0.018945,-0.007394,-0.006752


In [24]:
x_df = x_df.fillna(0)

In [25]:
print(x_df.shape)
print(x_df.columns)

(88891, 643)
Index(['gvkey', 'fyear', 'datadate', 'ACCHG', 'ACCRT', 'ACDO', 'ACO', 'ACODO',
       'ACOMINC', 'ACOX',
       ...
       'actual', 'CONSENSUS', 'IBESSHROUT', 'anndats', 'actual_p1',
       'consensus_p1', 'ibesshrout_p1', 'AFE_p1', 'AFE', 'accruals'],
      dtype='object', length=643)


In [26]:
x_df.dtypes

gvkey             object
fyear            float64
datadate         float64
ACCHG            float64
ACCRT            float64
                  ...   
consensus_p1     float64
ibesshrout_p1    float64
AFE_p1           float64
AFE              float64
accruals         float64
Length: 643, dtype: object

In [27]:
object_cols = []
col = 0
for dtype in x_df.dtypes:
    if (dtype != "float64"):
        object_cols.append(str(x_df.columns[col]))
    col += 1
print(object_cols)
x_df = x_df.drop(object_cols, axis=1)
print(x_df.dtypes)
print(x_df.shape)

['gvkey', 'conm', 'ffic12', 'anndats']
fyear            float64
datadate         float64
ACCHG            float64
ACCRT            float64
ACDO             float64
                  ...   
consensus_p1     float64
ibesshrout_p1    float64
AFE_p1           float64
AFE              float64
accruals         float64
Length: 639, dtype: object
(88891, 639)


In [28]:
y = x_df["AFE_p1"]
print(len(y))
x_df = x_df.drop(columns=["AFE_p1"])
X = x_df.to_numpy()
print(len(X))

88891
88891


In [29]:
model = LinearRegression()
model.fit(X, y)
r_sq = model.score(X, y)
print(r_sq)

0.6193770975888052


In [30]:
tuples = []
ind = 0
for coef in model.coef_:
    tuples.append((x_df.columns[ind], coef))
    ind += 1
tuples.sort(key = lambda x: abs(x[1]))
tuples.reverse()
for tup in tuples:
    print(tup)


('FLC', -95.71871529633549)
('FLMI', 34.59689777545227)
('CLD1', -33.37573607329067)
('AMROUFL', 30.808476720503187)
('FLNPV', 25.918441367036685)
('CLDTA', -21.551600269644695)
('OAFLCH', 21.445177312804283)
('WAVRFLT', -16.232646449486204)
('FLLT', 13.833391390360456)
('ROUAFLGR', -13.512508667190666)
('ROUAAM', -11.541218962839377)
('FIAFLCH', 11.423601999963376)
('FLMT', 10.837202206652652)
('XINTFL', 7.356746009005389)
('ROUAFLAM', -6.260952104947842)
('WAVFLR', -5.451868739576435)
('XINTLL', -3.0643914815237827)
('ROUAFLNT', -1.5533942466397894)
('AMROUL', 1.3536140222065396)
('PRCAD', -0.7821981595549761)
('PRCAEPS', 0.775079458674214)
('ROUAGR', 0.48252281215479176)
('OAOLCH', -0.28162027860331296)
('PNCID', 0.2468136922783025)
('OLMI', -0.24446403099109143)
('FIALP', 0.229351158509359)
('LLC', -0.22113148753713605)
('PNCAEPS', -0.21096620939551528)
('PNCAD', 0.207878413262446)
('PNCIEPS', -0.19475512389442634)
('PNCWID', -0.19379101157401463)
('PNCWIEPS', 0.18796232576232705)


In [40]:
#find best 5 features
from sklearn.feature_selection import SelectKBest, f_regression

temp = SelectKBest(score_func=f_regression, k=10)
temp2 = temp.fit_transform(X, y)
temp

/Users/adamkhakhar/opt/miniconda3/lib/python3.7/site-packages/sklearn/feature_selection/_univariate_selection.py:302: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/Users/adamkhakhar/.local/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1932: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


SelectKBest(score_func=<function f_regression at 0x1283a7f80>)

In [41]:
x_df.columns[temp.get_support()]

Index(['fyear', 'datadate', 'First_Date', 'actual', 'CONSENSUS', 'IBESSHROUT',
       'actual_p1', 'consensus_p1', 'ibesshrout_p1', 'AFE'],
      dtype='object')

In [38]:
X_new = x_df[x_df.columns[temp.get_support()]]
X_new

,fyear,actual,actual_p1,consensus_p1,AFE
0,1993.0,1.180,0.515,0.61000,-0.026103
1,1994.0,0.515,0.680,0.63000,-0.011111
2,1995.0,0.680,0.950,0.55000,0.005278
3,1996.0,0.950,0.550,0.55000,0.038571
4,2010.0,0.920,0.580,0.90000,-0.007394
...,...,...,...,...,...
88886,2012.0,-3.200,0.780,0.64300,-0.062292
88887,2013.0,0.780,0.140,0.56640,0.034745
88888,2014.0,0.140,-2.300,-0.93000,-0.045701
88889,2017.0,-8.660,-1.330,-1.40971,-0.000087


In [39]:
model = LinearRegression()
model.fit(X_new, y)
r_sq = model.score(X_new, y)
print(r_sq)

0.5410567575082544
